In [2]:
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dropout, Flatten, Dense
from tensorflow.keras.callbacks import EarlyStopping
import cv2
from PIL import Image
from keras.preprocessing.image import ImageDataGenerator
import glob
import keras.backend as K

K.clear_session() # 새로운 세션으로 시작

In [3]:
height = 224
width = 224

In [4]:
def img_read_resize(img_path):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (height, width))
    return img


In [5]:
def img_folder_read(root_dir):
    img_files = []
    nfiles = glob.glob(f"{root_dir}/*")
    nfiles = sorted(nfiles)
    for n_img in nfiles:
        try:
            img_files.append(img_read_resize(n_img))
        except:
            continue
    return img_files


In [7]:
test_root_dir = "archive/test"
test_root_dir

'archive/test'

In [10]:
data_generator = ImageDataGenerator(rescale = 1./255, validation_split=0.2)


In [16]:
train_generator = data_generator.flow_from_directory(
    directory = "archive/train",
    target_size = (height,width),
    batch_size = 10,
    class_mode = 'binary',
    subset = 'training',
    shuffle = False)

Found 22968 images belonging to 7 classes.


In [17]:
train_generator.class_indices


{'angry': 0,
 'disgust': 1,
 'fear': 2,
 'happy': 3,
 'neutral': 4,
 'sad': 5,
 'surprise': 6}

In [18]:
val_generator = data_generator.flow_from_directory(
    directory = "archive/train",
    target_size =(height, width),
    class_mode = 'binary',
    batch_size = 10,
    subset='validation',
    shuffle = False)

Found 5741 images belonging to 7 classes.


In [19]:
# vgg 사용
from tensorflow.keras.applications.vgg16 import VGG16
vgg = VGG16(weights = "imagenet", include_top=False, input_shape =(height, width, 3))

model = Sequential()
model.add(vgg)
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1,activation = "softmax"))

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2022-12-11 19:42:02.650491: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-12-11 19:42:02.651725: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


58889256/58889256 [==============================] - 1s 0us/step


In [20]:
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 1024)              25691136  
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 dense_1 (Dense)             (None, 1)                 1025      
                                                                 
Total params: 40,406,849
Trainable params: 40,406,849
Non-trainable params: 0
_________________________________________________________________


In [22]:
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics="accuracy")


In [23]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)


In [26]:
history = model.fit(train_generator, epochs = 5, callbacks = early_stop, validation_data = val_generator)


NameError: name 'scipy' is not defined